In [53]:
from pyspark import SparkContext

In [18]:
sc = SparkContext(master = 'local[*]', appName = 'BooksRating')

AVG of Books

In [198]:
BooksRating = sc.textFile('Book-Ratings.csv')

In [199]:
Split_BooksRating = BooksRating.map(lambda x : x.split(';')).filter(lambda x : x[0] != 'userid')

In [200]:
Split_BooksRating.count()

1149780

In [201]:
Split_BooksRating.take(5)

[['276725', '034545104X', '0'],
 ['276726', '0155061224', '5'],
 ['276727', '0446520802', '0'],
 ['276729', '052165615X', '3'],
 ['276729', '0521795028', '6']]

In [202]:
books_rate_avg = Split_BooksRating.map(lambda x : (x[1], int(x[2]))).groupByKey().mapValues(list)\
.map(lambda x : (x[0],sum(x[1])/len(x[1])))

In [203]:
books_rate_avg.take(5)

[('034545104X', 2.933333333333333),
 ('0155061224', 2.5),
 ('0521795028', 6.0),
 ('0600570967', 6.0),
 ('342310538', 5.0)]

(username; book title; username’ rate; Book Avg’ rate)

In [258]:
users = sc.textFile('Users.csv')
books = sc.textFile('Books.csv')

In [259]:
map_bookrates = BooksRating.map(lambda x : (x.split(';')[0],x.split(';')[1:]))

In [260]:
map_bookrates.take(5)

[('userid', ['isbn', 'rate']),
 ('276725', ['034545104X', '0']),
 ('276726', ['0155061224', '5']),
 ('276727', ['0446520802', '0']),
 ('276729', ['052165615X', '3'])]

In [261]:
map_users = users.map(lambda x : (x.split(';')[0],x.split(';')[1:]))

In [262]:
map_users.take(5)

[('UserID', ['USERNAME', 'Location', 'Age']),
 ('1', ['bzsufoRTLN2', 'nyc, new york, usa', 'NULL']),
 ('2', ['fq7kfHg4VEI', 'stockton, california, usa', '18']),
 ('3', ['W0Hbkd3xR8v', 'moscow, yukon territory, russia', 'NULL']),
 ('4', ['W51GahAx5Ap', 'porto, v.n.gaia, portugal', '17'])]

In [263]:
map_books = books.map(lambda x : (x.split(';')[0],x.split(';')[1:]))

In [264]:
map_books.take(2)

[('ISBN',
  ['BookTitle',
   'BookAuthor',
   'YearOfPublication',
   'Publisher',
   'ImageURLS',
   'ImageURLM',
   'ImageURLL']),
 ('0195153448',
  ['Classical Mythology',
   'Mark P. O. Morford',
   '2002',
   'Oxford University Press',
   'http://images.amazon.com/images/P/0195153448.01.THUMBZZZ.jpg',
   'http://images.amazon.com/images/P/0195153448.01.MZZZZZZZ.jpg',
   'http://images.amazon.com/images/P/0195153448.01.LZZZZZZZ.jpg'])]

In [265]:
user_rate_join = map_users.join(map_bookrates)

In [266]:
rate_user_join.take(5)

[('276725',
  (['034545104X', '0'], ['qP0KgArgnkK', 'tyler, texas, usa', 'NULL'])),
 ('276727',
  (['0446520802', '0'],
   ['AWJdCVnDLd8', 'h, new south wales, australia', '16'])),
 ('276744',
  (['038550120X', '7'], ['CoM54wt6AiU', 'torrance, california, usa', 'NULL'])),
 ('276746', (['0425115801', '0'], ['7uxi3eD6COS', 'fort worth, ,', 'NULL'])),
 ('276746', (['0449006522', '0'], ['7uxi3eD6COS', 'fort worth, ,', 'NULL']))]

In [267]:
user_book_rate = rate_user_join.filter(lambda x: x[1] is not None and x[1][0] is not None and x[1][1] is not None) \
                                .map(lambda x: (x[1][1][0], x[1][0][0], x[1][0][1])) \
                                .map(lambda x: (x[1], (x[0], x[2])))

In [268]:
user_book_rate.take(5)

[('034545104X', ('qP0KgArgnkK', '0')),
 ('0446520802', ('AWJdCVnDLd8', '0')),
 ('038550120X', ('CoM54wt6AiU', '7')),
 ('0425115801', ('7uxi3eD6COS', '0')),
 ('0449006522', ('7uxi3eD6COS', '0'))]

In [272]:
final_join = user_book_rate.join(books_rate_avg).join(map_books)

In [273]:
final_join.take(1)

[('0571058086',
  ((('KCOjXv3RxBK', '0'), 2.375),
   ['Waiting for Godot',
    'Samuel Beckett',
    '0',
    'Faber Faber Inc',
    'http://images.amazon.com/images/P/0571058086.01.THUMBZZZ.jpg',
    'http://images.amazon.com/images/P/0571058086.01.MZZZZZZZ.jpg',
    'http://images.amazon.com/images/P/0571058086.01.LZZZZZZZ.jpg']))]

In [292]:
final_rdd = final_join.map(lambda x : (x[1][0][0][0], x[1][1][0],x[1][0][0][1], x[1][0][1]))

In [293]:
final_rdd.take(10)

[('KCOjXv3RxBK', 'Waiting for Godot', '0', 2.375),
 ('CpOVZu1havb', 'Waiting for Godot', '10', 2.375),
 ('SELfAm7q9Iy', 'Waiting for Godot', '0', 2.375),
 ('d9jDCzsKYgM', 'Waiting for Godot', '0', 2.375),
 ('Ez0pZozqpv7', 'Waiting for Godot', '0', 2.375),
 ('aSMoX2Rk1zM', 'Waiting for Godot', '0', 2.375),
 ('I7ywUfQ6jdh', 'Waiting for Godot', '3', 2.375),
 ('ZC4Hk0GMggk', 'Waiting for Godot', '6', 2.375),
 ('PtKCXWxrZjk', 'False Memory', '10', 2.972972972972973),
 ('s8pY65csbH5', 'False Memory', '0', 2.972972972972973)]